# Projeto de TCC - People Analytics

In [3]:
!pip install autotime

     -------------------------------------- 46.8/46.8 KB 778.0 kB/s eta 0:00:00
     --------------------------------------- 11.1/11.1 MB 23.4 MB/s eta 0:00:00
     ------------------------------------- 294.9/294.9 KB 17.8 MB/s eta 0:00:00
     --------------------------------------- 12.9/12.9 MB 23.4 MB/s eta 0:00:00
     ---------------------------------------- 9.8/9.8 MB 27.3 MB/s eta 0:00:00
     ---------------------------------------- 8.1/8.1 MB 23.4 MB/s eta 0:00:00
     --------------------------------------- 14.8/14.8 MB 18.2 MB/s eta 0:00:00
     --------------------------------------- 12.6/12.6 MB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 608.6/608.6 KB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'

  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [8 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\d_dou\AppData\Local\Temp\pip-install-xvh6qtut\pyflux_c8c0a974cf174a798463515ff113250e\setup.py", line 101, in <module>
      setup_package()
    File "C:\Users\d_dou\AppData\Local\Temp\pip-install-xvh6qtut\pyflux_c8c0a974cf174a798463515ff113250e\setup.py", line 62, in setup_package
      from numpy.distutils.core import setup
  ModuleNotFoundError: No module named 'numpy'
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
You should consider up

In [4]:
%load_ext autotime

ModuleNotFoundError: No module named 'autotime'